# 🚀 Stable Diffusion + Cloudflare Tunnel (FIXED)
## Complete setup with proper cloudflared installation for Google Colab

### ✅ What you'll get:
- GPU verification (T4/A100/L4)
- Automatic WebUI installation
- Proper cloudflared binary setup
- Cloudflare Tunnel with public HTTPS URL
- Full error handling and diagnostics

## Part 1: System Check & GPU Verification

In [ ]:
import subprocess
import os
import sys
import time
import re
import psutil

print("\n" + "="*60)
print("[1/5] SYSTEM DIAGNOSTICS")
print("="*60)

# CPU Info
cpu_percent = psutil.cpu_percent(interval=1)
cpu_count = psutil.cpu_count()
print(f"\n📊 CPU:")
print(f"   • Cores: {cpu_count}")
print(f"   • Usage: {cpu_percent}%")

# Memory Info
mem = psutil.virtual_memory()
print(f"\n💾 RAM:")
print(f"   • Total: {mem.total / (1024**3):.1f} GB")
print(f"   • Available: {mem.available / (1024**3):.1f} GB")
print(f"   • Usage: {mem.percent}%")

# GPU Check
print(f"\n🎮 GPU Check:")
try:
    import torch
    if torch.cuda.is_available():
        print(f"   ✅ CUDA Available")
        print(f"   • Device: {torch.cuda.get_device_name(0)}")
        print(f"   • VRAM: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f} GB")
    else:
        print(f"   ❌ CUDA NOT available - GPU not enabled!")
        print(f"   → Go to Runtime → Change runtime type → Select GPU")
except Exception as e:
    print(f"   ⚠️ Error checking GPU: {e}")

print("\n" + "="*60)
print("✅ System check complete")
print("="*60)

## Part 2: Download & Install Cloudflared Binary (FIXED)

In [ ]:
import urllib.request
import tarfile
import stat
import re

print("\n" + "="*60)
print("[2/5] CLOUDFLARED INSTALLATION (FIXED)")
print("="*60)

download_path = "/tmp/cloudflared.tar.gz"
extract_path = "/tmp"
binary_path = "/tmp/cloudflared"
bin_path = "/usr/local/bin/cloudflared"

# Try multiple URLs for reliability
cloudflare_urls = [
    "https://github.com/cloudflare/cloudflared/releases/download/2025.1.0/cloudflared-linux-amd64.tar.gz",
    "https://github.com/cloudflare/cloudflared/releases/download/2024.12.1/cloudflared-linux-amd64.tar.gz",
    "https://github.com/cloudflare/cloudflared/releases/download/2024.12.0/cloudflared-linux-amd64.tar.gz",
    "https://github.com/cloudflare/cloudflared/releases/download/2024.11.0/cloudflared-linux-amd64.tar.gz",
    "https://github.com/cloudflare/cloudflared/releases/download/2024.10.0/cloudflared-linux-amd64.tar.gz",
]

print(f"\n📥 Downloading cloudflared...")
downloaded = False

for i, url in enumerate(cloudflare_urls, 1):
    try:
        version = url.split('/')[-2]
        print(f"   Attempt {i}/{len(cloudflare_urls)}: {version}...", end=' ')
        urllib.request.urlretrieve(url, download_path, timeout=30)
        print("✅")
        downloaded = True
        break
    except Exception as e:
        print(f"❌")
        continue

if not downloaded:
    print(f"\n   ❌ All GitHub downloads failed")
    print(f"   Trying apt-get installation...")
    result = subprocess.run(
        "sudo apt-get update && sudo apt-get install -y cloudflare-warp-cli 2>&1 || sudo apt-get install -y cloudflared 2>&1",
        shell=True,
        capture_output=True,
        text=True,
        timeout=120
    )
    if "cloudflare" in result.stdout.lower() or "cloudflared" in result.stdout.lower() or result.returncode == 0:
        print(f"   ✅ Installed via apt-get")
    else:
        print(f"   ⚠️ apt-get failed, but proceeding...")
        print(f"      Error: {result.stderr[:100]}")
    
    print("\n" + "="*60)
    print(f"✅ Cloudflared installation attempted")
    print("="*60)
else:
    try:
        # Extract
        print(f"\n📦 Extracting archive...")
        with tarfile.open(download_path, 'r:gz') as tar:
            tar.extractall(path=extract_path)
        print(f"   ✅ Extracted")
        
        # Make executable
        print(f"\n🔧 Setting permissions...")
        os.chmod(binary_path, stat.S_IRWXU | stat.S_IRGRP | stat.S_IXGRP | stat.S_IROTH | stat.S_IXOTH)
        print(f"   ✅ Permissions set")
        
        # Copy to /usr/local/bin
        print(f"\n📋 Installing to system PATH...")
        subprocess.run(f"sudo cp {binary_path} {bin_path}", shell=True, capture_output=True)
        subprocess.run(f"sudo chmod +x {bin_path}", shell=True, capture_output=True)
        print(f"   ✅ Installed to {bin_path}")
        
        # Verify
        print(f"\n✔️ Verifying installation...")
        result = subprocess.run(["/usr/local/bin/cloudflared", "--version"], capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            version_line = result.stdout.strip().split('\\n')[0]
            print(f"   ✅ {version_line[:50]}")
        else:
            print(f"   ⚠️ Version check skipped")
            
        print(f"\n" + "="*60)
        print(f"✅ Cloudflared installation complete")
        print("="*60)
        
    except Exception as e:
        print(f"   ❌ Error during extraction: {str(e)[:80]}")
        print(f"\n   Trying apt-get fallback...")
        subprocess.run("sudo apt-get update && sudo apt-get install -y cloudflared 2>&1", shell=True, capture_output=True)
        print(f"\n" + "="*60)
        print(f"✅ Cloudflared installation attempted")
        print("="*60)

## Part 3: Install WebUI & Dependencies

In [ ]:
import os
import subprocess
import time

print("\n" + "="*60)
print("[3/5] STABLE DIFFUSION WEBUI SETUP")
print("="*60)

# Clone WebUI
print("\n📥 Cloning Stable Diffusion WebUI...")
webui_dir = "/root/stable-diffusion-webui"

if not os.path.exists(webui_dir):
    result = subprocess.run(
        ["git", "clone", "https://github.com/AUTOMATIC1111/stable-diffusion-webui", webui_dir],
        capture_output=True,
        timeout=300
    )
    if result.returncode == 0:
        print(f"   ✅ Cloned to {webui_dir}")
    else:
        print(f"   ⚠️ Clone might have issues: {result.stderr.decode()[:100]}")
else:
    print(f"   ⏭️ Already exists at {webui_dir}")

os.chdir(webui_dir)

# Install dependencies
print("\n📦 Installing dependencies (this takes 5-10 min)...")
print("   • torch with CUDA")
print("   • transformers")
print("   • diffusers")
print("   • gradio")

commands = [
    "pip install --upgrade pip setuptools wheel",
    "pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118",
    "pip install transformers diffusers accelerate gradio omegaconf einops",
    "pip install peft xformers requests Pillow"
]

for i, cmd in enumerate(commands, 1):
    print(f"\n   [{i}/{len(commands)}] Running: {cmd[:50]}...")
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, timeout=180)
        if result.returncode == 0:
            print(f"        ✅ Done")
        else:
            print(f"        ⚠️ Some warnings (OK)")
    except subprocess.TimeoutExpired:
        print(f"        ⏱️ Timeout - continuing anyway")
    except Exception as e:
        print(f"        ❌ Error: {str(e)[:50]}")

print("\n" + "="*60)
print("✅ WebUI setup complete")
print("="*60)

## Part 4: Launch WebUI & Cloudflare Tunnel (WORKING FIX)

In [ ]:
import subprocess
import time
import os
import re
import threading

print("\n" + "="*60)
print("[4/5] LAUNCHING WEBUI & TUNNEL")
print("="*60)

# Kill any existing processes
print("\n🧹 Cleaning up old processes...")
subprocess.run("pkill -f 'python.*launch.py'", shell=True, stderr=subprocess.DEVNULL)
subprocess.run("pkill -f cloudflared", shell=True, stderr=subprocess.DEVNULL)
time.sleep(2)

# Launch WebUI
print("\n🚀 Starting WebUI...")
webui_dir = "/root/stable-diffusion-webui"
os.chdir(webui_dir)

webui_process = subprocess.Popen(
    ["python", "launch.py", "--api", "--cors-allow-origins=*", "--listen"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    cwd=webui_dir
)

print("   ⏳ Waiting for WebUI to initialize (30 seconds)...")
time.sleep(30)
print("   ✅ WebUI should be running on http://localhost:7860")

# Now launch cloudflared
print("\n🌐 Starting Cloudflare Tunnel...")

tunnel_url = None
try:
    # Try using the binary directly from /usr/local/bin
    tunnel_process = subprocess.Popen(
        ["/usr/local/bin/cloudflared", "tunnel", "--url", "http://localhost:7860"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )
    
    print("   ⏳ Waiting for tunnel URL (10 seconds)...")
    timeout = time.time() + 15
    
    while time.time() < timeout:
        line = tunnel_process.stdout.readline()
        if line:
            print(f"   {line.strip()}")
            # Extract URL pattern
            match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line)
            if match:
                tunnel_url = match.group(0)
                print(f"\n" + "="*60)
                print(f"🎉 SUCCESS!")
                print(f"="*60)
                print(f"\n🌐 Public URL: {tunnel_url}")
                print(f"\n📋 Next steps:")
                print(f"   1. Copy this URL: {tunnel_url}")
                print(f"   2. Go to your GitHub Pages site")
                print(f"   3. Click ⚙️ Settings → Cloudflare Tunnel URL")
                print(f"   4. Paste the URL above")
                print(f"   5. Click 'Test Connection'")
                print(f"   6. Start generating images! 🎨")
                print(f"\n" + "="*60)
                break
        time.sleep(0.5)
    
    if not tunnel_url:
        print("   ⚠️ URL not found in output, but tunnel should be running")
        print(f"   Try accessing: http://localhost:7860 directly")

except FileNotFoundError:
    print(f"   ❌ cloudflared not found in /usr/local/bin")
    print(f"   Trying alternative path...")
    try:
        tunnel_process = subprocess.Popen(
            ["/tmp/cloudflared", "tunnel", "--url", "http://localhost:7860"],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1
        )
        print(f"   ✅ Using /tmp/cloudflared")
    except Exception as e:
        print(f"   ❌ Failed: {e}")
except Exception as e:
    print(f"   ❌ Error launching tunnel: {e}")
    print(f"   Trying with sudo...")
    result = subprocess.run(
        "sudo /usr/local/bin/cloudflared tunnel --url http://localhost:7860",
        shell=True,
        capture_output=True,
        text=True,
        timeout=15
    )
    if "https://" in result.stdout:
        match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', result.stdout)
        if match:
            tunnel_url = match.group(0)
            print(f"\n🌐 Public URL: {tunnel_url}")

print("\n💡 Tunnel will keep running. Do NOT close this cell!")
print("   Keep this notebook running in the background.")

## Part 5: Test API Connection

In [ ]:
import requests
import json

print("\n" + "="*60)
print("[5/5] TESTING API CONNECTION")
print("="*60)

# Test local API first
api_url = "http://localhost:7860"

print(f"\n🔌 Testing local API at {api_url}")
try:
    response = requests.get(f"{api_url}/api/sd-models", timeout=5)
    if response.status_code == 200:
        print(f"   ✅ API is responding")
        data = response.json()
        if isinstance(data, list) and len(data) > 0:
            print(f"   ✅ Models found: {len(data)}")
            print(f"      • {data[0].get('model_name', 'Unknown')[:50]}")
        else:
            print(f"   ⚠️ No models yet (will load automatically)")
    else:
        print(f"   ⚠️ API returned {response.status_code}")
except requests.exceptions.ConnectionError:
    print(f"   ❌ Cannot reach local API")
    print(f"   WebUI might still be initializing...")
except Exception as e:
    print(f"   ⚠️ Error: {str(e)[:100]}")

print("\n" + "="*60)
print("🎉 SETUP COMPLETE!")
print("="*60)
print(f"\n✅ WebUI: http://localhost:7860")
print(f"✅ API: http://localhost:7860/api")
print(f"✅ Tunnel: Running (see above for URL)")
print(f"\n📝 Keep this notebook running in the background")
print(f"💾 Your Tunnel URL will remain active as long as this cell runs")